In [1]:
# 디시
# 멀티 프로세싱 이용 x
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

# 검색 결과 페이지 URL 템플릿
base_url = "https://search.dcinside.com/post/p/{}/sort/latest/q/.EC.95.84.EB.B0.98.EB.96.BC"

# HTTP 요청 헤더 (사이트 차단 방지)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 크롤링할 페이지 개수 설정 (원하는 만큼 조절 가능)
max_pages = 10  # 테스트를 위해 3페이지만 크롤링 (더 늘려도 됨)

# 결과를 저장할 리스트
results = []

# 여러 페이지 순회
for page in range(1, max_pages + 1):
    print(f"현재 {page} 페이지 크롤링 중...")

    # 페이지별 검색 URL 생성
    search_url = base_url.format(page)

    # 페이지 요청
    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        print(f"페이지 {page} 요청 실패 (상태 코드: {response.status_code})")
        break

    soup = BeautifulSoup(response.text, 'html.parser')

    # 게시글 목록 추출
    posts = soup.select("ul.sch_result_list li")

    # 게시글이 없으면 종료
    if not posts:
        print("더 이상 게시글이 없습니다. 크롤링 종료.")
        break

    for post in posts:
        # 게시글 제목과 URL
        title_tag = post.find("a", class_="tit_txt")
        if title_tag:
            title = title_tag.get_text(strip=True)
            post_url = title_tag["href"]
        else:
            continue

        # 게시글 상세 페이지 요청
        post_response = requests.get(post_url, headers=headers)
        if post_response.status_code != 200:
            print(f"게시글 {post_url} 요청 실패, 건너뜀")
            continue

        post_soup = BeautifulSoup(post_response.text, 'html.parser')

        # 게시글 내용
        content_tag = post_soup.find("div", class_="write_div")
        content = content_tag.get_text(strip=True) if content_tag else "내용 없음"

        # 게시글 날짜
        date_tag = post_soup.find("span", class_="gall_date")
        date_str = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
        try:
            post_date = datetime.strptime(date_str, "%Y.%m.%d %H:%M:%S")
            post_date = post_date.strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            post_date = "날짜 변환 실패"

        # 추천 수
        recommend_tag = post_soup.find("p", class_="up_num")
        recommend_count = recommend_tag.get_text(strip=True) if recommend_tag else "0"

        # 비추천 수
        dislike_tag = post_soup.find("p", class_="down_num")
        dislike_count = dislike_tag.get_text(strip=True) if dislike_tag else "0"

        # 댓글 수
        comment_tag = post_soup.find("span", id=lambda x: x and x.startswith("comment_total"))
        comment_count = comment_tag.get_text(strip=True) if comment_tag else "0"
        
        # 조회수
        view_count_tag = post_soup.find("span", class_="gall_count")
        view_count = view_count = view_count_tag.get_text(strip=True).replace("조회 ", "") if view_count_tag else "0"

        # 작성자 닉네임
        nickname_tag = post_soup.find("span", class_="nickname")
        nickname = nickname_tag.get_text(strip=True) if nickname_tag else "익명"

        # 작성자 IP
        ip_tag = post_soup.find("span", class_="ip")
        ip = ip_tag.get_text(strip=True).replace("(", "").replace(")", "") if ip_tag else "IP 없음"

        # 결과 저장
        results.append({
            "url": post_url,
            "date": post_date,
            "content": content,
            "author": f"{nickname} ({ip})",  # 작성자 + IP 추가
            "views": view_count,
            "recommend": recommend_count,
            "dislike": dislike_count,
            "comments": comment_count
        })

        # 요청 사이 딜레이 추가 (과부하 방지)
        time.sleep(1)

# DataFrame으로 변환
df = pd.DataFrame(results)

# CSV 파일로 저장
csv_filename = "dcinside_avante_detailed_posts_2.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"크롤링 완료! {csv_filename} 파일이 생성되었습니다.")

/Users/hxxjaewoong17/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


현재 1 페이지 크롤링 중...
현재 2 페이지 크롤링 중...
현재 3 페이지 크롤링 중...
현재 4 페이지 크롤링 중...
현재 5 페이지 크롤링 중...
현재 6 페이지 크롤링 중...
현재 7 페이지 크롤링 중...
현재 8 페이지 크롤링 중...
현재 9 페이지 크롤링 중...
현재 10 페이지 크롤링 중...
크롤링 완료! dcinside_avante_detailed_posts_2.csv 파일이 생성되었습니다.
